In [1]:
## Preloads
using Statistics
using FFTW
using Plots
using BenchmarkTools
using Profile
using LinearAlgebra
using Measures
using HDF5
push!(LOAD_PATH, pwd())
using DHC_2DUtils
using MLDatasets
using Images
theme(:juno)
using DSP
using Interpolations
using Distributed
using ProgressMeter

  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:162
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:162
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:166
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:166
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:176
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:179
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:181
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:184
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:186
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:189
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:203
  likely near /n/home12/saydjari/.julia/packages/Plots

In [2]:
using ImageCore
using Colors

In [10]:
# read the MNIST training data, x=image, y=class
train_x, train_y = CIFAR10.traindata();
test_x, test_y   = CIFAR10.testdata();

In [11]:
lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x)[4]
    s = StackedView(transpose(train_x[:,:,1,i]),transpose(train_x[:,:,2,i]),transpose(train_x[:,:,3,i]));
    temp = Images.Gray.(colorview(RGB, s))
    timg = convert(Array{Float64}, temp)
    push!(lst_train,timg)
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x)[4]
    s = StackedView(transpose(test_x[:,:,1,i]),transpose(test_x[:,:,2,i]),transpose(test_x[:,:,3,i]));
    temp = Images.Gray.(colorview(RGB, s))
    timg = convert(Array{Float64}, temp)
    push!(lst_test,timg)
end

In [4]:
M = 100
angle_array = []
for i = 2π/M:2π/M:2π
    append!(angle_array,i)
end

M = 100
train_angles = [2π/M,π/3,2π/3,3π/3,4π/3,5π/3];

In [3]:
workers()

1-element Array{Int64,1}:
 1

In [4]:
addprocs(30)

30-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
  ⋮
 20
 21
 22
 23
 24
 25
 26
 27
 28
 29
 30
 31

In [15]:
@everywhere begin
    using Statistics
    using LinearAlgebra
    using Distributed
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using Images
    using Interpolations
    using DSP
    using ProgressMeter
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=128,wd=2)
    
    function wind_2d(nx)
        dx   = nx/2-1
        filter = zeros(Float64, nx, nx)
        A = DSP.tukey(nx, 0.3)
        itp = extrapolate(interpolate(A,BSpline(Linear())),0)
        @inbounds for x = 1:nx
            sx = x-dx-1    # define sx,sy so that no fftshift() needed
            for y = 1:nx
                sy = y-dx-1
                r  = sqrt.((sx).^2 + (sy).^2) + nx/2
                filter[x,y] = itp(r)
            end
        end
        return filter
    end
    
    function mnist_pad(im; θ=0.0)
        imbig = imresize(im,(64,64))
        mu_imbig = mean(imbig)
        imbig .-= mu_imbig
        imbig .*= wind_2d(64)
        impad = zeros(Float64,128,128)
        impad[96:-1:33,33:96] = imbig

        if θ != 0.0
            imrot = imrotate(impad, θ, axes(impad), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot .+ mu_imbig
        end

        return impad.+ mu_imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash,filter_hash)
        return WST
    end
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, Iterators.product(angle_array,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

Progress: 100%|█████████████████████████████████████████| Time: 1:18:24
HDF5-DIAG: Error detected in HDF5 (1.12.0) thread 0:
  #000: H5F.c line 705 in H5Fcreate(): unable to create file
    major: File accessibility
    minor: Unable to open file
  #001: H5VLcallback.c line 3393 in H5VL_file_create(): file create failed
    major: Virtual Object Layer
    minor: Unable to create file
  #002: H5VLcallback.c line 3358 in H5VL__file_create(): file create failed
    major: Virtual Object Layer
    minor: Unable to create file
  #003: H5VLnative_file.c line 65 in H5VL__native_file_create(): unable to create file
    major: File accessibility
    minor: Unable to open file
  #004: H5Fint.c line 1568 in H5F_open(): unable to open file: name = '../scratch_AKS/cifar10_test_angles.h5', tent_flags = 13
    major: File accessibility
    minor: Unable to open file
  #005: H5FD.c line 741 in H5FD_open(): open failed
    major: Virtual File Layer
    minor: Unable to initialize object
  #006: H5FDsec

ErrorException: Error creating file ../scratch_AKS/cifar10_test_angles.h5

In [16]:
h5write("cifar10_test_angles.h5", "main/data", mnist_DHC_out)

In [17]:
# read the MNIST training data, x=image, y=class
train_x, train_y = CIFAR100.traindata();
test_x, test_y   = CIFAR100.testdata();

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x)[4]
    s = StackedView(transpose(train_x[:,:,1,i]),transpose(train_x[:,:,2,i]),transpose(train_x[:,:,3,i]));
    temp = Images.Gray.(colorview(RGB, s))
    timg = convert(Array{Float64}, temp)
    push!(lst_train,timg)
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x)[4]
    s = StackedView(transpose(test_x[:,:,1,i]),transpose(test_x[:,:,2,i]),transpose(test_x[:,:,3,i]));
    temp = Images.Gray.(colorview(RGB, s))
    timg = convert(Array{Float64}, temp)
    push!(lst_test,timg)
end

This program has requested access to the data dependency CIFAR100.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Dataset: The CIFAR-100 dataset
Authors: Alex Krizhevsky, Vinod Nair, Geoffrey Hinton
Website: https://www.cs.toronto.edu/~kriz/cifar.html
Reference: https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf

[Krizhevsky, 2009]
    Alex Krizhevsky.
    "Learning Multiple Layers of Features from Tiny Images",
    Tech Report, 2009.

The CIFAR-100 dataset is a labeled subsets of the 80
million tiny images dataset. It consists of 60000
32x32 colour images in 100 classes. Specifically, it
has 100 classes containing 600 images each. There are
500 training images and 100 testing images per class.
The 100 classes in the CIFAR-100 are grouped into 20
superclasses. Each image comes with a "fine" label
(the class to which it belongs) and a "coarse" label
(the superclass to which it belongs).

The compressed archive f

stdin>  y


In [18]:
@everywhere begin
    using Statistics
    using LinearAlgebra
    using Distributed
    push!(LOAD_PATH, "/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/main/")
    using DHC_2DUtils
    using FFTW
    using Images
    using Interpolations
    using DSP
    using ProgressMeter
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=128,wd=2)
    
    function wind_2d(nx)
        dx   = nx/2-1
        filter = zeros(Float64, nx, nx)
        A = DSP.tukey(nx, 0.3)
        itp = extrapolate(interpolate(A,BSpline(Linear())),0)
        @inbounds for x = 1:nx
            sx = x-dx-1    # define sx,sy so that no fftshift() needed
            for y = 1:nx
                sy = y-dx-1
                r  = sqrt.((sx).^2 + (sy).^2) + nx/2
                filter[x,y] = itp(r)
            end
        end
        return filter
    end
    
    function mnist_pad(im; θ=0.0)
        imbig = imresize(im,(64,64))
        mu_imbig = mean(imbig)
        imbig .-= mu_imbig
        imbig .*= wind_2d(64)
        impad = zeros(Float64,128,128)
        impad[96:-1:33,33:96] = imbig

        if θ != 0.0
            imrot = imrotate(impad, θ, axes(impad), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot .+ mu_imbig
        end

        return impad.+ mu_imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash,filter_hash)
        return WST
    end
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, Iterators.product(angle_array,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("cifar100_test_angles.h5", "main/data", mnist_DHC_out)

Progress: 100%|█████████████████████████████████████████| Time: 0:58:20


In [5]:
M = 50
angle_array = []
for i = 2π/M:2π/M:2π
    append!(angle_array,i)
end

In [20]:
L = 8
rand_mat = rand(512,512)
filter_hash = fink_filter_hash(1,L,nx=512,wd=2)
temp = DHC_compute(rand_mat,filter_hash,filter_hash);

In [6]:
train_x, train_y = MNIST.traindata()
test_x, test_y   = MNIST.testdata()

lst_train = Array{Any}(undef, 0)
for i = 1:60000
    push!(lst_train,train_x[:,:,i])
end


lst_test = Array{Any}(undef, 0)
for i = 1:10000
    push!(lst_test,test_x[:,:,i])
end

In [9]:
M = 50
train_angles = [2π/M,π/3,2π/3,3π/3,4π/3,5π/3];

In [7]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,16,nx=128,wd=2,pc=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, Iterators.product(angle_array,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_test_ang_1_1_wd2_pc2.h5", "main/data", mnist_DHC_out)

Progress: 100%|█████████████████████████████████████████| Time: 2:59:01


In [11]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,16,nx=128,wd=2,pc=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, Iterators.product(train_angles,lst_train))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_train_ang_1_1_wd2_pc2.h5", "main/data", mnist_DHC_out)

Progress: 100%|█████████████████████████████████████████| Time: 1:09:01
